In [8]:
%matplotlib inline

import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from numpy import mean
from math import sqrt,acos,cos,sin,pi,exp,log,isnan,atan2
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from numpy import asarray
from root_pandas import read_root
from matplotlib import gridspec
from scipy import stats
 
from datetime import date

datafolder = '/home/nwkamp/Research/MicroBooNE/Data'
sysfolder = '/home/nwkamp/Research/MicroBooNE/Data/systematics'
auxfolder = '/home/nwkamp/Research/MicroBooNE/1L1PSelection/aux'
dumpfolder = 'PlotDumps'

RSE  = ['run','subrun','event']
RSEV = ['run','subrun','event','vtxid']
RSECV = ['run','subrun','event',"isCV"]

#with open('/home/dcianci/Physics/1e1p/1mu1pSelection/bdtweight_series4_Sept16_run1.pickle','rb') as handle: bkgBDT_run1 = pickle.load(handle)          # Load BDT weights for 1mu1p background differentiation     
#with open('/home/dcianci/Physics/1e1p/1mu1pSelection/bdtweight_series4_Sept16_run2.pickle','rb') as handle: bkgBDT_run2 = pickle.load(handle)          # Load BDT weights for 1mu1p background differentiation  
#with open('/home/dcianci/Physics/1e1p/1mu1pSelection/bdtweight_series4_Sept16_run3.pickle','rb') as handle: bkgBDT_run3 = pickle.load(handle)          # Load BDT weights for 1mu1p background differentiation  
    
#with open('/home/dcianci/Physics/1e1p/testzone/1L1PSelection/bdt_models/bdtweight_series2_june1_run1.pickle','rb') as handle: bkgBDT_run1 = pickle.load(handle)          # Load BDT weights for 1mu1p background differentiation     
#with open('/home/dcianci/Physics/1e1p/testzone/1L1PSelection/bdt_models/bdtweight_series2_june1_run3.pickle','rb') as handle: bkgBDT_run3 = pickle.load(handle)          # Load BDT weights for 1mu1p background differentiation  
    
vars_jan21 = ['Phis','ChargeNearTrunk','Enu_1e1p','PhiT_1e1p','AlphaT_1e1p','PT_1e1p','PTRat_1e1p','BjXB_1e1p','BjYB_1e1p','SphB_1e1p','Q0_1e1p','Q3_1e1p','Lepton_PhiReco','Lepton_TrackLength','Proton_PhiReco','Proton_ThetaReco']
#training_varbs = [x.Enu_1e1p,x.Electron_Edep,x.PT_1e1p,x.AlphaT_1e1p,x.SphB_1e1p,x.PzEnu_1e1p,x.ChargeNearTrunk,x.Q0_1e1p,x.Q3_1e1p,x.Thetas,x.Phis,x.PTRat_1e1p,x.Proton_TrackLength,x.Lepton_TrackLength,x.Proton_ThetaReco,x.Proton_PhiReco,x.Lepton_ThetaReco,x.Lepton_PhiReco,max(x.MinShrFrac,-1),max(x.MaxShrFrac,-1),x.shower1_smallQ_Y/(x.shower1_sumQ_Y+1e-6) ,x.BjX_1e1p,x.BjY_1e1p]    
bdtVars = vars_jan21

In [9]:
def bless_MC_labels(row):
    mclabel = ''
    intlabel = ''
    parentlabel = ''
    pizero = [1090,1086,1090,1080,1015,1013,1011,1008,1006,1004]
    piplusminus = [1085,1079,1032,1017,1014,1007,1005,1003,1028,1021,1016,1012,1010,1009]
        
    if abs(row['nu_pdg']) == 12:
        intlabel = 'nue'
    elif abs(row['nu_pdg']) == 14:
        intlabel = 'numu'
        
    if not (row['MC_nproton']==1 and row['MC_nlepton']==1):
        return 'nLmP'
    elif not 0 < row['MC_scedr'] <= 5.0:
        return 'offvtx'
    elif not abs((row['MC_energyInit']-row['Enu_1e1p'])/row['MC_energyInit']) < 0.2:
        return 'badreco'    
    else:
        if row['nu_interaction_type'] == 1001:
            mclabel = 'CCQE'
        elif row['nu_interaction_type'] == 1000:
            mclabel = 'MEC'
        elif row['nu_interaction_type'] in pizero:
            mclabel = 'pizero'
        elif row['nu_interaction_type'] in piplusminus:
            mclabel = 'piplusminus' 
        else:
            mclabel = 'other'
    return '%s_%s'%(intlabel,mclabel)

def bless_int_labels(row):
    mclabel = ''
    intlabel = ''
    parentlabel = ''
    pizero = [1090,1086,1090,1080,1015,1013,1011,1008,1006,1004]
    piplusminus = [1085,1079,1032,1017,1014,1007,1005,1003,1028,1021,1016,1012,1010,1009]
        
    if abs(row['nu_pdg']) == 12:
        intlabel = 'nue'
    elif abs(row['nu_pdg']) == 14:
        intlabel = 'numu'
        
    if row['nu_interaction_type'] == 1001:
        mclabel = 'CCQE'
    elif row['nu_interaction_type'] == 1000:
        mclabel = 'MEC'
    elif row['nu_interaction_type'] in pizero:
        mclabel = 'pizero'
    elif row['nu_interaction_type'] in piplusminus:
        mclabel = 'piplusminus' 
    else:
        mclabel = 'other'
           
    return '%s_%s'%(intlabel,mclabel)

In [10]:
tag = date.today()

# Tag dictionary:
# Oct21_cmt - all up to date as of the collaboration meeting in october. Identical selection to internal note, now with training samples marked

# All Precuts
orthogonalcut = 'MaxShrFrac > .2'
precuts ='PassSimpleCuts == 1 and PassShowerReco == 1 and Proton_Edep > 60 and \
          Electron_Edep > 35 and Proton_ThetaReco < %3.3f and _pi0mass , 50' % (np.pi/2)
pmtprecuts = 'TotPE > 20 and PorchTotPE < 20'
s_precut = orthogonalcut + ' and ' + precuts + ' and ' + pmtprecuts

In [11]:
def proc_df(df_dlana,df_wgts,df_goodrun,bdtwgts,isMC=True):
    
    # join with goodruns list and cv weights list
    df_full = df_dlana.join(df_goodrun.set_index('run'),on='run')
    if(isMC):
        df_full = df_full.join(df_wgts.set_index(RSE)[['nu_interaction_mode','nu_interaction_type','xsec_corr_weight','spline_weight','nu_interaction_ccnc','nu_pdg','nu_energy_true']],on=RSE)
    df_full_goodruns = df_full.query('good==1')   # keep good runs
    df_full_goodruns_precuts = df_full_goodruns.query(s_precut)    # apply precuts
    
    
    if(isMC):
        df_full_goodruns_precuts.insert(0,'mc_label',df_full_goodruns_precuts.apply(bless_MC_labels,axis=1))
        df_full_goodruns_precuts.insert(0,'int_label',df_full_goodruns_precuts.apply(bless_int_labels,axis=1))
       
    # add a bunch of helpful variables!
    df_full_goodruns_precuts.insert(0,'MPID_eminus',[ef[2] for ef in df_full_goodruns_precuts['EminusPID_int_v'].values])
    df_full_goodruns_precuts.insert(0,'MPID_muon',[ef[2] for ef in df_full_goodruns_precuts['MuonPID_int_v'].values])
    df_full_goodruns_precuts.insert(0,'MPID_proton',[ef[2] for ef in df_full_goodruns_precuts['ProtonPID_int_v'].values])
    df_full_goodruns_precuts.insert(0,'MPID_gamma',[ef[2] for ef in df_full_goodruns_precuts['GammaPID_int_v'].values])
    df_full_goodruns_precuts.insert(0,'MPID_pion',[ef[2] for ef in df_full_goodruns_precuts['PionPID_int_v'].values])
    df_full_goodruns_precuts.insert(0,'Lepton_CosTheta',np.cos(df_full_goodruns_precuts['Lepton_ThetaReco'].values).tolist())
    df_full_goodruns_precuts.insert(0,'Proton_CosTheta',np.cos(df_full_goodruns_precuts['Proton_ThetaReco'].values).tolist())
    
    
    # add most current bdt weights.
    df_full_goodruns_precuts.insert(0,'BDTscore_1e1p',bdtwgts.predict_proba(df_full_goodruns_precuts[bdtVars].values.tolist())[:,0])
    df_full_nodupes = df_full_goodruns_precuts.sort_values('BDTscore_1e1p',descending=True).drop_duplicates(RSE).sort_index()
    
    return df_full_nodupes

def proc_df_fakedata(df_dlana,bdtwgts):
    
    df_full_precuts = df_dlana.query(s_precut)    # apply precuts
    
    # add a bunch of helpful variables!
    df_full_precuts.insert(0,'MPID_eminus',[ef[2] for ef in df_full_precuts['EminusPID_int_v'].values])
    df_full_precuts.insert(0,'MPID_muon',[ef[2] for ef in df_full_precuts['MuonPID_int_v'].values])
    df_full_precuts.insert(0,'MPID_proton',[ef[2] for ef in df_full_precuts['ProtonPID_int_v'].values])
    df_full_precuts.insert(0,'MPID_gamma',[ef[2] for ef in df_full_precuts['GammaPID_int_v'].values])
    df_full_precuts.insert(0,'MPID_pion',[ef[2] for ef in df_full_precuts['PionPID_int_v'].values])
    df_full_precuts.insert(0,'Lepton_CosTheta',np.cos(df_full_precuts['Lepton_ThetaReco'].values).tolist())
    df_full_precuts.insert(0,'Proton_CosTheta',np.cos(df_full_precuts['Proton_ThetaReco'].values).tolist())
    
    # add most current bdt weights.
    df_full_goodruns_precuts.insert(0,'BDTscore_1e1p',bdtwgts.predict_proba(df_full_goodruns_precuts[bdtVars].values.tolist())[:,0])
    df_full_nodupes = df_full_goodruns_precuts.sort_values('BDTscore_1e1p',descending=True).drop_duplicates(RSE).sort_index()
        
    return df_full_nodupes

In [12]:
# Time to load the good runs list
good_run1_df = pd.read_csv('%s/pass_r1.txt'%auxfolder)
good_run2_df = pd.read_csv('%s/pass_r2.txt'%auxfolder)
good_run3_df = pd.read_csv('%s/pass_r3.txt'%auxfolder)

good_run1_df['good'] = 1
good_run2_df['good'] = 1
good_run3_df['good'] = 1
#print(good_run1_df)

# Training

In [6]:
# Are we in  the training sample?
a_train_run1 = np.genfromtxt('bdt_run1_trainsample.csv',delimiter=',')
a_train_run3 = np.genfromtxt('bdt_run3_trainsample.csv',delimiter=',')
dic_train_run1 = {}
dic_train_run3 = {}

for tr in a_train_run1:
    idx = tuple((tr[0],tr[1]))
    dic_train_run1[idx] = dict(intrain=1)

for tr in a_train_run3:
    idx = tuple((tr[0],tr[1]))
    dic_train_run3[idx] = dict(intrain=1)
    
def InTrainRun1(row):
    try:
        return dic_train_run1[row.run,row.subrun]['intrain']
    except:
        return 0
    
def InTrainRun3(row):
    try:
        return dic_train_run3[row.run,row.subrun]['intrain']
    except:
        return 0

In [7]:
# Beam quality
beamq_df = read_root('%s/beamdataquality_remix_bnb5e19.root'%datafolder,'bdq')

# MC BNB OVERLAY

##  Run1

In [6]:
s_mc = 'numu_run1'
df_mc = read_root('%s/mcc9_bnb_nu_overlay_CV_run1_500k_numu_v1_1_3_fvv.root'%datafolder,'dlana/FinalVertexVariables')
df_mc_cvweight = read_root('%s/weights_forCV_v48_Sep24_bnb_nu_DetVar_run1.root'%auxfolder)

#df_mc.insert(0,'InTraining',df_mc.apply(InTrainRun1,axis=1)) # If you are training

df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run1_df,0,True) 
#df_mc_full = df_mc_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes if using nue intrinsic
#print(list(df_mc_full))
#df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

In [10]:
s_mc = 'numu_lowe_run1'
df_mc = read_root('%s/bnb_overlay/mcc9_v29e_run1_bnb_nu_overlay_LowE.root'%datafolder,'dlana/FinalVertexVariables')
df_mc_cvweight = read_root('%s/bnb_overlay/weights_forCV_v48_Sep24_bnb_nu_lowE_run1.root'%datafolder)

df_mc.insert(0,'InTraining',df_mc.apply(InTrainRun1,axis=1))

df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run1_df,bkgBDT_run1,True)
df_mc_full = df_mc_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

##  Run 2

In [58]:
# s_mc = 'numu_run2'
# df_mc = read_root('%s/bnb_overlay/mcc9_v29e_dl_run2_bnb_nu_overlay_finalbdt.root'%datafolder,'dlana/FinalVertexVariables')
# df_mc_cvweight = read_root('%s/bnb_overlay/weights_forCV_v40_bnb_nu_run2.root'%datafolder)

# df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run2_df,bkgBDT_run3,True) 
# df_mc_full = df_mc_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes

# df_mc_full['InTraining'] = 0

# print('Total Verts',len(df_mc_full))
# print('mec',len(df_mc_full.query("mc_label in ['numu_MEC']")))
# print('piplusminus',len(df_mc_full.query("mc_label in ['numu_piplusminus']")))
# print('pizero',len(df_mc_full.query("mc_label in ['numu_pizero']")))
# print('numu CCQE',len(df_mc_full.query("mc_label in ['numu_CCQE']")))
# print('other',len(df_mc_full.query("mc_label in ['numu_other','BNB nue_other','nue_MEC','nue_piplusminus','nue_pizero']")))

# df_mc_full.query('abs(MC_energyInit-nu_energy_true) < .0003').to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

# #del df_mc, df_mc_cvweight, df_mc_full

Total Verts 35569
mec 209
piplusminus 229
pizero 87
numu CCQE 1963
other 159


## Run3

In [7]:
s_mc = 'numu_run3'
df_mc = read_root('%s/mcc9_v40_bnb_nu_overlay_run3b_CV_v1_1_3_fvv_numu.root'%datafolder,'dlana/FinalVertexVariables')
df_mc_cvweight = read_root('%s/weights_forCV_v48_Sep24_bnb_nu_DetVar_run3.root'%auxfolder)

#df_mc.insert(0,'InTraining',df_mc.apply(InTrainRun3,axis=1))

df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run3_df,0,True) 
#df_mc_full = df_mc_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes if using nu_e int

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

In [12]:
s_mc = 'numu_lowe_run3'
df_mc = read_root('%s/bnb_overlay/mcc9_v29e_run3b_bnb_nu_overlay_LowE.root'%datafolder,'dlana/FinalVertexVariables')
df_mc_cvweight = read_root('%s/bnb_overlay/weights_forCV_v48_Sep24_bnb_nu_lowE_run3.root'%datafolder)

df_mc.insert(0,'InTraining',df_mc.apply(InTrainRun3,axis=1))

df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run3_df,bkgBDT_run3,True) 
df_mc_full = df_mc_full.query('not (nu_interaction_ccnc==0 and abs(nu_pdg)==12)') # cut out nue ccqes

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

# MC NUE OVERLAY

In [13]:
s_mc = 'nue_run1'
df_mc = read_root('%s/nue_intrinsic_overlay/mcc9_v28_wctagger_nueintrinsics_stripped.root'%datafolder,'FinalVertexVariables')
df_mc_cvweight = read_root('%s/nue_intrinsic_overlay/weights_forCV_v48_Sep24_intrinsic_nue_run1.root'%datafolder)

df_mc.insert(0,'InTraining',df_mc.apply(InTrainRun1,axis=1))

df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run1_df,bkgBDT_run1,True)

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

In [56]:
# s_mc = 'nue_run2'
# df_mc = read_root('%s/nue_intrinsic_overlay/mcc9_v29e_dl_run2_bnb_intrinsics_nue_overlay_finalbdt.root'%datafolder,'dlana/FinalVertexVariables')
# df_mc_cvweight = read_root('%s/nue_intrinsic_overlay/weights_forCV_v40_intrinsic_nue_run2.root'%datafolder)

# df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run2_df,bkgBDT_run3,True)

# df_mc_full['InTraining'] = 0

# #df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

# #del df_mc, df_mc_cvweight, df_mc_full

In [14]:
s_mc = 'nue_run3'
df_mc = read_root('%s/nue_intrinsic_overlay/mcc9_v29e_run3b_bnb_intrinsic_nue_overlay_nocrtremerge_stripped.root'%datafolder,'FinalVertexVariables')
df_mc_cvweight = read_root('%s/nue_intrinsic_overlay/weights_forCV_v48_Sep24_intrinsic_nue_run3.root'%datafolder)

df_mc.insert(0,'InTraining',df_mc.apply(InTrainRun3,axis=1))

df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run3_df,bkgBDT_run3,True)

df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

del df_mc, df_mc_cvweight, df_mc_full

# Dirt

In [15]:
# s_mc = 'dirt_run1'
# df_mc = read_root('%s/dirt/FVV-Prime-dirt-Mar3-WC-1e1p.root'%datafolder,'FinalVertexVariables')
# df_mc_cvweight = read_root('%s/dirt/weights_forCV_v40_dirt_nu_run1.root'%datafolder)

# df_mc_full = proc_df(df_mc,df_mc_cvweight,good_run1_df,bkgBDT_run1,True)

# df_mc_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_mc,tag))

# del df_mc, df_mc_cvweight, df_mc_full

# EXT

In [16]:
s_ext = 'ext_run1'
df_ext = read_root('%s/ext/mcc9_v28_wctagger_extbnbFULL_stripped.root'%datafolder)

df_ext.insert(0,'InTraining',df_ext.apply(InTrainRun1,axis=1))

df_ext_full = proc_df(df_ext,'',good_run1_df,bkgBDT_run1,False)

df_ext_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_ext,tag))

In [17]:
s_ext = 'ext_run3'
df_ext = read_root('%s/ext/mcc9_v29e_dl_run3_G1_extbnb_stripped.root'%datafolder)
    
df_ext.insert(0,'InTraining',df_ext.apply(InTrainRun3,axis=1))

df_ext_full = proc_df(df_ext,'',good_run3_df,bkgBDT_run3,False)

df_ext_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_ext,tag)) 

# Data

In [21]:
s_data = 'data_run1_5e19'
df_data = read_root('%s/bnb/mcc9_v28_wctagger_5e19.root'%datafolder,'dlana/FinalVertexVariables')

# add beam quality filter
df_data = df_data.join(beamq_df.set_index(['run','subrun','event']),on=['run','subrun','event'])
df_data_full = proc_df(df_data,'',good_run1_df,bkgBDT_run1,False)

df_data_full['InTraining'] = 0

df_data_full.query('result==1').to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [22]:
s_data = 'data_run3_1e19'
df_data = read_root('%s/bnb/mcc9_v28_wctagger_run3_bnb1e19.root'%datafolder,'dlana/FinalVertexVariables')

df_data_full = proc_df(df_data,'',good_run3_df,bkgBDT_run3,False)

df_data_full['InTraining'] = 0

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [23]:
s_data = 'data_run2_filter'
df_data = read_root('%s/bnb/mcc9_v29e_run2_D2E1_1e1p_fvv.root'%datafolder,'dlana/FinalVertexVariables')

df_data_full = proc_df(df_data,'',good_run2_df,bkgBDT_run2,False)

df_data_full['InTraining'] = 0

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [24]:
s_data = 'data_run3_filter'
df_data = read_root('%s/bnb/mcc9_v29e_run3_F1G1_1e1p_fvv.root'%datafolder,'dlana/FinalVertexVariables')

df_data_full = proc_df(df_data,'',good_run3_df,bkgBDT_run3,False)

df_data_full['InTraining'] = 0

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [25]:
s_data = 'data_run1_filter'
df_data = read_root('%s/bnb/mcc9_v29e_dl_run1_C1_bnb_dlfilter_1e1p_v1_1_2b_fvv.root'%datafolder,'dlana/FinalVertexVariables')

df_data_full = proc_df(df_data,'',good_run1_df,bkgBDT_run1,False)

df_data_full['InTraining'] = 0

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [10]:
s_data = 'fakedata_set1_run1'
df_data = read_root('%s/fakedata/dlfilter_fakedata_v08_00_00_29e_dl_ubdlana_v1_1_2_set1_run1_1e1p_stripped.root'%datafolder,'FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run1)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [11]:
s_data = 'fakedata_set1_run3b'
df_data = read_root('%s/fakedata/dlfilter_fakedata_v08_00_00_29e_dl_ubdlana_v1_1_2_set1_run3b_1e1p_stripped.root'%datafolder,'FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run3)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [13]:
s_data = 'fakedata_set3_run1'
df_data = read_root('%s/fakedata/mcc9_v29e_dl_set3_fakedata_run1_numu_v1_1_3_fvv.root'%datafolder,'dlana/FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run1)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [14]:
s_data = 'fakedata_set3_run3b'
df_data = read_root('%s/fakedata/mcc9_v29e_dl_set3_fakedata_run3b_numu_v1_1_3_fvv.root'%datafolder,'dlana/FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run3)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [7]:
s_data = 'fakedata_set2_run1'
df_data = read_root('%s/fakedata/mcc9_v29e_dl_set2_fakedata_run1_numu_v1_1_3_fvv.root'%datafolder,'dlana/FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run1)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [8]:
s_data = 'fakedata_set2_run3b'
df_data = read_root('%s/fakedata/mcc9_v29e_dl_set2_fakedata_run3b_numu_v1_1_3_fvv.root'%datafolder,'dlana/FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run3)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [20]:
s_data = 'fakedata_set4_run1'
df_data = read_root('%s/fakedata/mcc9_v29e_dl_set4_fakedata_run1_numu_v1_1_3_fvv.root'%datafolder,'dlana/FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run1)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [21]:
s_data = 'fakedata_set4_run3b'
df_data = read_root('%s/fakedata/mcc9_v29e_dl_set4_fakedata_run3b_numu_v1_1_3_fvv.root'%datafolder,'dlana/FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run3)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

In [22]:
s_data = 'fakedata_set5_run1'
df_data = read_root('%s/fakedata/mcc9_v29e_dl_set5_fakedata_run1_numu_v1_1_3_fvv.root'%datafolder,'dlana/FinalVertexVariables')

# remember to turn off pmt precuts
df_data_full = proc_df_fakedata(df_data,bkgBDT_run1)

df_data_full.to_parquet('%s/pickles/%s_%s.parquet'%(datafolder,s_data,tag))

# Detector Systematics (run3; not updated)

# example of one detvar


In [9]:
df_cv = read_root('%s/mcc9_v40a_dl_run3b_bnb_intrinsic_nue_overlay_CV.root'%datafolder,'dlana/FinalVertexVariables')
df_newr = pd.read_csv('/home/nwkamp/Research/MicroBooNE/1e1pSelection/RooUnfold/SystematicsSelection/SystematicsSelection_Run3_CV_StandardLowHigh.txt',delimiter=', ')
df_cv = df_cv.merge(df_newr,how='right').drop_duplicates(RSE)

df_sce = read_root('%s/mcc9_v40a_dl_run3b_bnb_intrinsic_nue_overlay_recomb2.root'%datafolder,'dlana/FinalVertexVariables')
df_newr = pd.read_csv('/home/nwkamp/Research/MicroBooNE/1e1pSelection/RooUnfold/SystematicsSelection/SystematicsSelection_Run3_recomb2_StandardLowHigh.txt',delimiter=', ')
df_sce = df_sce.merge(df_newr,how='right').drop_duplicates(RSE)


/home/nwkamp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/nwkamp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [17]:
dvkeys = ['standard','low','high']
sRSE = ['sample'] + RSE
sRSECV = ['sample'] + RSECV

df_mc_detvarweight_r1_standard = read_root('%s/weights_forCV_v48_Sep24_intrinsic_nue_DetVar_run1.root'%auxfolder).drop_duplicates(RSE)
df_mc_detvarweight_r1_standard.insert(0,'sample',[dvkeys[0] for i in range(len(df_mc_detvarweight_r1_standard))])
df_mc_detvarweight_r1_low = read_root('%s/weights_forCV_v48_Sep24_eLEE_low_DetVar_run1.root'%auxfolder).drop_duplicates(RSE)
df_mc_detvarweight_r1_low.insert(0,'sample',[dvkeys[1] for i in range(len(df_mc_detvarweight_r1_low))])
df_mc_detvarweight_r1_high = read_root('%s/weights_forCV_v48_Sep24_eLEE_high_DetVar_run1.root'%auxfolder).drop_duplicates(RSE)
df_mc_detvarweight_r1_high.insert(0,'sample',[dvkeys[2] for i in range(len(df_mc_detvarweight_r1_high))])
df_mc_detvarweight_r1 = pd.concat((df_mc_detvarweight_r1_standard,df_mc_detvarweight_r1_low,df_mc_detvarweight_r1_high)).drop_duplicates(sRSE)


df_mc_detvarweight_r3_standard = read_root('%s/weights_forCV_v48_Sep24_intrinsic_nue_DetVar_run3.root'%auxfolder).drop_duplicates(RSE)
df_mc_detvarweight_r3_standard.insert(0,'sample',[dvkeys[0] for i in range(len(df_mc_detvarweight_r3_standard))])
df_mc_detvarweight_r3_low = read_root('%s/weights_forCV_v48_Sep24_eLEE_low_DetVar_run3.root'%auxfolder).drop_duplicates(RSE)
df_mc_detvarweight_r3_low.insert(0,'sample',[dvkeys[1] for i in range(len(df_mc_detvarweight_r3_low))])
df_mc_detvarweight_r3_high = read_root('%s/weights_forCV_v48_Sep24_eLEE_high_DetVar_run3.root'%auxfolder).drop_duplicates(RSE)
df_mc_detvarweight_r3_high.insert(0,'sample',[dvkeys[2] for i in range(len(df_mc_detvarweight_r3_high))])
df_mc_detvarweight_r3 = pd.concat((df_mc_detvarweight_r3_standard,df_mc_detvarweight_r3_low,df_mc_detvarweight_r3_high)).drop_duplicates(sRSE)

def getFullDetvarDF_1e1p(dv,run,POTCV,POTDV):
    
    if run=='1':
        df_goodrun = good_run1_df
        df_wgts = df_mc_detvarweight_r1
    elif run=='3':
        df_goodrun = good_run3_df
        df_wgts = df_mc_detvarweight_r3
    else:
        print('Select either run 1 or run 3! Exiting...')
        return -1
    
    df_cv = pd.read_csv('/home/nwkamp/Research/MicroBooNE/1e1pSelection/RooUnfold/SystematicsSelection/SystematicsSelection_Run%s_CV_StandardLowHigh.txt'%run,delimiter=', ')
    df_dv = pd.read_csv('/home/nwkamp/Research/MicroBooNE/1e1pSelection/RooUnfold/SystematicsSelection/SystematicsSelection_Run%s_%s_StandardLowHigh.txt'%(run,dv),delimiter=', ')
    
    df_cv['isCV'] = True
    df_dv['isCV'] = False
    #df_cv['POT']  = sum(POTCV.values())
    #df_dv['POT'] = sum(POTDV.values())
    df_cv['POT']  = df_cv.apply(lambda row: POTCV['standard'] + int(row.Etrue<400)*POTCV['low'] + int(row.Etrue>400 and row.Etrue<800)*POTCV['high'], axis=1)
    df_dv['POT'] = df_dv.apply(lambda row: POTDV['standard'] + int(row.Etrue<400)*POTDV['low'] + int(row.Etrue>400 and row.Etrue<800)*POTDV['high'], axis=1)

    

    df_full = pd.concat((df_cv,df_dv))
    print('df_full = %d  cv = %d , dv = %d'%(df_full.shape[0],df_cv.shape[0],df_dv.shape[0]))
    df_full = df_full.join(df_wgts.set_index(sRSE)[['nu_interaction_mode','nu_interaction_type','xsec_corr_weight','spline_weight','nu_interaction_ccnc','nu_pdg','nu_energy_true']],on=sRSE)
    df_full_wGoodruns = df_full.join(df_goodrun.set_index('run'),on='run')
    df_full_goodruns = df_full_wGoodruns.query('good == 1 and Enu_1e1p > 0')
    print('df_full_goodruns = %d  cv = %d , dv = %d'%(df_full_goodruns.shape[0],df_full_goodruns.query("isCV==True").shape[0],df_full_goodruns.query("isCV==False").shape[0]))

    #df_full_goodruns.insert(0,'PassPrecuts1e1p',df_full_goodruns.apply(passPrecut,axis=1))     

    df_full_nodupes = df_full_goodruns.sort_values('sigProb',ascending=True).drop_duplicates(sRSECV).sort_index()
    print('df_full_nodupes = %d  cv = %d , dv = %d'%(df_full_nodupes.shape[0],df_full_nodupes.query("isCV==True").shape[0],df_full_nodupes.query("isCV==False").shape[0]))

    return df_full_goodruns

# all detvar with thier correspondance cv 


In [18]:
run1POT = {
    "CV": {'standard':2.40236914697e+22,'low':4.317567545E+23,'high':7.062363279E+22},
    "LYRayleigh": {'standard':2.35130607446e+22,'low':4.838409135E+23,'high':7.870279156E+22},
    "LYDown": {'standard':2.87664943915e+22,'low':4.418290404E+23,'high':7.363032075E+22},
    "LYAtt" : {'standard':2.92293960383e+22,'low':4.477090683E+23,'high':7.723968494E+22}
}
run3POT = {
    "CV": {'standard':2.89633486917e+22,'low':4.420872429E+23,'high':7.40598415E+22},
    "LYRayleigh": {'standard':3.12607092491e+22,'low':5.124870339E+23,'high':8.3075838217E+22},
    "LYDown": {'standard':3.02224657246e+22,'low':4.664119568E+23,'high':7.82781203E+22},
    "LYAtt" : {'standard':5.59852627949E+22,'low':4.968635449E+23,'high':8.51917546E+22},
    "recomb2" : {'standard':3.60888588249E+22,'low':4.109640309E+23,'high':5.71869841E+22},
    "SCE" : {'standard':2.87085754999e+22,'low':4.294431989E+23,'high':7.62076123E+22},
    "WireModX" : {'standard':3.20900856709e+22,'low':5.080818329E+23,'high':8.62560656E+22},
    "WireModYZ" : {'standard':2.54766720272e+22,'low':4.437820079E+23,'high':7.39558079E+22},
    "WireModThetaXZ" : {'standard':2.44011517946e+22,'low':4.544662239E+23,'high':7.50850899E+22},
    "WireModThetaYZ" : {'standard':1.95821121697e+22,'low':4.537001309E+23,'high':7.96152227E+22}
}

tag = 'v0'

run = '1'
for dv in run1POT.keys():
    #print(good_run3_df)
    df_bnb_wCV_wPass_wProc = getFullDetvarDF_1e1p(dv,run,run1POT['CV'],run1POT[dv])
    df_bnb_wCV_wPass_wProc.to_parquet('%s/pickles/%s_run%s_%s.parquet'%(sysfolder,dv,run,tag))
    print('done %s, %i events'%(dv,df_bnb_wCV_wPass_wProc.shape[0]))
    
run = '3'
for dv in run3POT.keys():
    #print(good_run3_df)
    df_bnb_wCV_wPass_wProc = getFullDetvarDF_1e1p(dv,run,run3POT['CV'],run3POT[dv])
    df_bnb_wCV_wPass_wProc.to_parquet('%s/pickles/%s_run%s_%s.parquet'%(sysfolder,dv,run,tag))
    print('done %s, %i events'%(dv,df_bnb_wCV_wPass_wProc.shape[0]))



/home/nwkamp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:34: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/home/nwkamp/.local/lib/python2.7/site-packages/ipykernel_launcher.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


df_full = 7834  cv = 3713 , dv = 4121
df_full_goodruns = 7834  cv = 3713 , dv = 4121
df_full_nodupes = 7834  cv = 3713 , dv = 4121
done LYAtt, 7834 events
df_full = 7592  cv = 3713 , dv = 3879
df_full_goodruns = 7592  cv = 3713 , dv = 3879
df_full_nodupes = 7592  cv = 3713 , dv = 3879
done LYRayleigh, 7592 events
df_full = 7426  cv = 3713 , dv = 3713
df_full_goodruns = 7426  cv = 3713 , dv = 3713
df_full_nodupes = 7426  cv = 3713 , dv = 3713
done CV, 7426 events
df_full = 7779  cv = 3713 , dv = 4066
df_full_goodruns = 7779  cv = 3713 , dv = 4066
df_full_nodupes = 7779  cv = 3713 , dv = 4066
done LYDown, 7779 events
df_full = 5752  cv = 3052 , dv = 2700
df_full_goodruns = 5752  cv = 3052 , dv = 2700
df_full_nodupes = 5752  cv = 3052 , dv = 2700
done WireModThetaYZ, 5752 events
df_full = 6027  cv = 3052 , dv = 2975
df_full_goodruns = 6027  cv = 3052 , dv = 2975
df_full_nodupes = 6027  cv = 3052 , dv = 2975
done WireModThetaXZ, 6027 events
df_full = 6517  cv = 3052 , dv = 3465
df_full_goo

# Run1

# DIRT